<h1>Proton Therapy Optimization</h1>

<h3>Naive modeling</h3>

The first idea that comes to mind is to make a straight forward mathematical translation of the oral problem description. That would be to set a vector of *i* integer variables *x[i]*, representing the number of proton fractions that should be performed to patient *i*. 

We would then symbolize our input matrix as *c[i,j]*: the benefit of offering *j* proton fractions to patient *i*. While this is a very intuitive model it suffers from a very non linear object function. In the simplest case where we would like to optimize the sum or average BED, the object functions to be maximized would be:

**f = Σ c[i, x[i]] over all i's**

<h3>A better modeling idea</h3>


Instead we would prefer to go with a linear model. For that reason we could select another set of decision variables, that is a binary matrix *x[i,j]* where a value of 1 denotes that *j* fractions should be performed on patient *i*. Obviously we should constraint every rows sum to 1 (a patient will receive 0,1, ... or N fractions).

Our object function can now be expressed as elegandly as:

**f = Σ c[i,j] * x[i,j] over all j's over all i's**

This is obviously as linear as it gets. Lets code it!

In [22]:
# GUROBI must be installed and a licence file must be discoverable in one of the default locations
from gurobipy import *

# Get some additional dependencies
import pandas as pd
import numpy as np

In [23]:
# Read the dataset into a 2d array. benefit[i,j] is the benefit patient i will get from j fractions
data = pd.read_csv('data/PayoffMatrix.txt', delim_whitespace = True)
benefit = data.values

In [24]:
# Lets represent our decisions as indexes
num_patients, max_fractions_per_patient = benefit.shape
patients = [i for i in range(num_patients)]
fractions = [j for j in range(max_fractions_per_patient)]

In [25]:
m = Model('proton_therapy')

x = m.addVars(num_patients, max_fractions_per_patient, vtype = GRB.BINARY)

# Only one choice of fractions per patient is valid
m.addConstrs(quicksum(x[i,j] for j in fractions) == 1 for i in patients)

# We can only perform so many proton therapies per week
CAPACITY = 100
m.addConstr(quicksum(quicksum(x[i,j] * fractions[j] for j in fractions) for i in patients) <= CAPACITY)

m.setObjective(quicksum(x[i,j] * benefit[i,j] for i in patients for j in fractions), GRB.MAXIMIZE)
m.setParam('OutputFlag', 0)
m.update()
m.optimize()

In [26]:
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr("x", x)
    for i in patients:
        optimum = -1
        for j in fractions:
            if(solution[i,j] == 1):
                print("Patient " + str(i) + " should receive " + str(j) + " fractions")
                break

            

Patient 0 should receive 10 fractions
Patient 1 should receive 3 fractions
Patient 2 should receive 4 fractions
Patient 3 should receive 0 fractions
Patient 4 should receive 0 fractions
Patient 5 should receive 6 fractions
Patient 6 should receive 9 fractions
Patient 7 should receive 10 fractions
Patient 8 should receive 12 fractions
Patient 9 should receive 7 fractions
Patient 10 should receive 4 fractions
Patient 11 should receive 4 fractions
Patient 12 should receive 9 fractions
Patient 13 should receive 15 fractions
Patient 14 should receive 4 fractions
Patient 15 should receive 3 fractions
